In [1]:
# This cell is not needed if you have pip installed topologicpy
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")

In [3]:
import numpy as np
from topologicpy.GA import GA  # your wrapper

# 2D decision vector (2 genes), both in [-3, 3].
gene_space = [{"low": -3.0, "high": 3.0}] * 2

# --- 2-objective fitness (MAXIMIZE both) ---
# f1 peaks at (+1.5, -1.0)
# f2 peaks at (-1.5, +1.0)
# These conflict, producing a clear Pareto front.
def mo2_fitness(ga_instance, solution, solution_idx):
    x = np.asarray(solution, dtype=float)

    # Objective 1: close to a target point A
    a = np.array([ 1.5, -1.0], dtype=float)
    f1 = -np.sum((x - a)**2)

    # Objective 2: close to a target point B
    b = np.array([-1.5,  1.0], dtype=float)
    f2 = -np.sum((x - b)**2)

    return [f1, f2]


ga = GA(
    num_genes=2,
    gene_space=gene_space,
    fitness_function=mo2_fitness,
    parent_selection_type="nsga2",   # multi-objective
    sol_per_pop=80,
    num_generations=60,
    num_parents_mating=25,
    crossover_type="single_point",
    mutation_type="random",
    mutation_percent_genes=50,       # for 2 genes, ensure >=1 gene mutates
    random_seed=42,
)

summary = ga.Run()
print(summary)
print("Pareto size:", len(ga.ParetoFrontIndices()))

fig = ga.PlotParetoFront(
    title="2-Objective Pareto Front",
    objective_names=["toward A", "toward B"],
    show_all_points=True,
    pareto_color="darkred",
    show_grid=True,
    show_legend = True,
    grid_color="lightgrey",
    grid_width=0.5
)
fig.show()


GARunSummary(generations_completed=60, best_solution=array([-1.50110071,  0.93206999]), best_fitness=array([-1.27394999e+01, -4.61569779e-03]), best_index=0)
Pareto size: 33


In [6]:
import numpy as np

from topologicpy.GA import GA
# --- Example 4D -> 4-objective multi-objective problem (maximize all) ---
# Decision vector x has 4 genes in [-2, 2].
# Objectives are shaped so the Pareto set is broad:
#   f1 prefers x near (+1, +1, +1, +1)
#   f2 prefers x near (-1, -1, -1, -1)
#   f3 prefers small |x| (near 0)
#   f4 prefers large |x| (near corners)
#
# This creates inherent conflicts => a non-trivial Pareto front.
#
# Note: PyGAD expects MAXIMIZATION. All objectives below are higher-is-better.

def mo_fitness(ga_instance, solution, solution_idx):
    x = np.asarray(solution, dtype=float)

    # f1: peak at +1 vector (negative squared distance)
    f1 = -np.sum((x - 1.0)**2)

    # f2: peak at -1 vector
    f2 = -np.sum((x + 1.0)**2)

    # f3: peak at 0 vector
    f3 = -np.sum(x**2)

    # f4: prefers large magnitude (but bounded); use +sum(x^2) so it conflicts with f3
    f4 = np.sum(x**2)

    return [f1, f2, f3, f4]


# --- Gene space for 4D vector in [-2, 2] ---
gene_space = [{"low": -2.0, "high": 2.0}] * 4

ga = GA(
    num_genes=4,
    gene_space=[{"low": -2.0, "high": 2.0}] * 4,
    fitness_function=mo_fitness,
    parent_selection_type="nsga2",
    sol_per_pop=120,
    num_generations=200,
    num_parents_mating=40,
    mutation_percent_genes=25,
    random_seed=42,
)

# ga.EnableCheckpointing(
#     directory="C:/Users\sarwj\OneDrive - Cardiff University/Desktop/checkpoints",
#     interval=10,
#     keep_last=5,
# )

# Optionally resume if there is an existing checkpoint:
# ga.ResumeFromLatestCheckpoint()

summary = ga.Run()
print(summary)


# --- Pareto front indices + count ---
pf_idx = ga.ParetoFrontIndices()
print("Pareto front size:", len(pf_idx))

# --- Plot Pareto with Plotly ---
# With 4 objectives, this will generate a parallel-coordinates plot (per the wrapper).
fig = ga.PlotParetoFront(
    title="4-Objective Pareto (Parallel Coordinates)",
    objective_names=["toward +1", "toward -1", "toward 0", "large magnitude"],
    show_all_points=True,
    width=1200,
    height=600,
    max_points=1000,
    pareto_color="red",
    show_legend=True,
    show_grid=True,
    grid_color="lightgrey",
    grid_width=0.5
)
fig.show()


GARunSummary(generations_completed=200, best_solution=array([1.90087545, 1.87965211, 1.84877436, 1.98025972]), best_fitness=array([ -3.26669146, -33.70493804, -14.48581475,  14.48581475]), best_index=0)
Pareto front size: 120
